In [11]:
import re
import ast
import sys
import csv
import math
import nltk
import random
import pandas as pd
from math import ceil
from math import floor
from io import StringIO

In [12]:
#appending print statement if not 
def ch_print(s):
    
    with open("t.txt",mode="w+") as t:
        t.write(s)                    #writing the code into file 
        t.seek(0)                     #changing pointers location back to begining
        a=t.readlines()               
    
    e=re.findall(r'print',a[-1])      #checking for the print statement in the last line 

    if not e:
        if "=" in a[-1]:              #ex. if s= "ab=2+3"
            d=''
            for i in range(a[-1].index('=')):
                if a[-1][i] !=" ":
                    d+=a[-1][i]    
        else:
            d=a[-1]

        with open("t.txt",mode="a+") as t:
            t.write(f'\nprint({d})')   #appending the print statement---appending "print(ab)"
            t.seek(0)
            a=t.read()
        return a                           #return the new code
    
    else:
        return s                          

In [13]:
# Extract Source code from the string
def extract_source_code_from_string(string):
    # Split the string by newline
    if '?' in string:
        string = string.split('?')# Remove the first line
        string = string[1:]# Join the string
        string = ''.join(string)   
    elif ';' in string:
        string = string.split(';')
        string = string[1:]
        string = ';'.join(string)
    elif ':' in string:
        string = string.split(':')
        string = string[1:]
        string = ':'.join(string)
    
    # Clearing the string    
    string=re.sub(">>> |>>>", "",string)
    string=re.sub('""', '"',string)

    string=string.strip() 
    
    string=ch_print(string) # Check for print , not present then append it into the code 
    
    return string  #return the string

In [14]:
#returning the ouput of the code
def compiler(s):
    code =f'''{s}'''
    #print(code)
    old_stdout = sys.stdout
    redirected_output = sys.stdout = StringIO()
    exec(compile(code, filename="", mode="exec"))
    sys.stdout = old_stdout
    return redirected_output.getvalue()

In [15]:
def isnumeric(num):
    try:
        float(num)
        return True
    except ValueError:
        return False

In [16]:
def num_generator(num):
    all_choice=[]
    all_choice.append(num)
    j=3
    while j:
        j-=1
        wrong_choice=''
        for i in num:
            if i!='-' and i!=".":
                wrong_choice+=str(random.randint(0,9))
            else:
                wrong_choice+=i
        if wrong_choice not in all_choice:
            all_choice.append(wrong_choice)
        else:
            j+=1
    random.shuffle(all_choice)
    ind=all_choice.index(num)
    options="ABCD"
    for i in range(4):
        all_choice[i]=f'({options[i]})' +all_choice[i]
    return all_choice+[num]+["("+options[ind]+')']

In [17]:
def char_generator(char):
    if char.islower():
        return chr(random.randint(ord('a'), ord('z')))
    else:
        return chr(random.randint(ord('A'), ord('Z')))

In [18]:
def choice_generator(num):
    all_choice=[]
    all_choice.append(num)
    change=0
    if "\\n"in num and "\n" in num:
        num=num.replace('\n',':')
        num=num.replace("\\n",'\n')
        change=1
    elif "\\n" in num:
        num=num.replace('\\n','\n')
        change=1
    
    j=3
    while j:
        j-=1
        wrong_choice=""
        for i in num:
            if isnumeric(i):
                wrong_choice+=str(random.randint(0,9))
            elif ord(i)>=ord('a') and ord(i)<=ord('z'):
                wrong_choice+=char_generator(i)
            else:
                wrong_choice+=i
        if wrong_choice not in all_choice:
            if change:
                wrong_choice=wrong_choice.replace("\n",'\\n')
                wrong_choice=wrong_choice.replace(':','\n')
            all_choice.append(wrong_choice)
        else:
            j+=1   
    if change:
        num=num.replace("\n",'\\n')
        num=num.replace(':','\n')
    random.shuffle(all_choice)
    ind=all_choice.index(num)
    options="ABCD"
    for i in range(4):
        all_choice[i]=f'({options[i]})' +all_choice[i]
    return all_choice+[num]+["("+options[ind]+')']

In [19]:
if __name__ == '__main__':
    # Extract from File
    data=pd.read_csv("data.csv")
    # Extract the source code
    data['source_code'] = data['Question'].apply(extract_source_code_from_string)
    
    lst_data=list(data['source_code'])      
    for j,i in enumerate(lst_data):        
        print(f'\n---->{j}--{i} \n ===>',end=' ')
        code =f'''{i}'''
        exec(compile(code, filename="", mode="exec"))
        print('\n')
    output_data=[]
    
    for i in lst_data:
        enter=compiler(i)
        if isnumeric(enter) and "." in enter:
            output_data.append(str(round(float(enter),3)))    
        else:
            output_data.append(enter)        #storing the output in a list

    for i in range(len(output_data)):        
        output_data[i]=output_data[i][:-1]     #cleaning output 
    final_output=[]
    for i in output_data:
        if isnumeric(i):
            final_output.append(num_generator(i))
        else:
            final_output.append(choice_generator(i))
    
    for i in range(len(final_output)):
        final_output[i]=[data["Question"][i]] + final_output[i]
    header = ['Question', '(a)', '(b)', '(c)','(d)','Correct Answer',"Correct Option"]
    
    
    for i in range(len(final_output)):
        print(i,'Q.',end=" ")
        for j in final_output[i]:
            print(j)
        print('\n')    
    with open('Generated_Option.csv', 'w', encoding='UTF8') as f:
        writer = csv.writer(f)
        # write the header
        writer.writerow(header)
        for i in final_output:
            writer.writerow(i)
            


---->0--print (r"\njavat\npoint") 
 ===> \njavat\npoint



---->1--X = 2+9*((3*12)-8)/10
print(X) 
 ===> 27.2



---->2--x = int(43.55+2/2)
print(x) 
 ===> 44



---->3--24//6%3, 24//4//2
print(24//6%3, 24//4//2) 
 ===> 1 3



---->4--x=float(4+int(2.39)%2)
x
print(x) 
 ===> 4.0



---->5--print("a"+"bc") 
 ===> abc



---->6--print("abcd"[2:]) 
 ===> cd



---->7--str1 = 'hello'
str2 = ','
str3 = 'world'
print(str1[-1:]) 
 ===> o



---->8--print (r"\nhello") 
 ===> \nhello



---->9--print('new' 'line') 
 ===> newline



---->10--print('new' 'line') 
 ===> newline



---->11--str1="helloworld"
print(str1[::-1]) 
 ===> dlrowolleh



---->12--print(0xA + 0xB + 0xC) 
 ===> 33



---->13--print('abcdef12'.replace('cd', '12')) 
 ===> ab12ef12



---->14--names1 = ['Amir', 'Bala', 'Chales']
 
if 'amir' in names1:
    print(1)
else:
    print(2) 
 ===> 2



---->15--names1 = ['Amir', 'Bala', 'Charlie']
names2 = [name.lower() for name in names1]
 
print(names2[2][0]) 
 ===> c



---->16--nu

In [20]:
x= 5+5

In [23]:
print(x)

10
